In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

burgers_loss_correction = 0.37741058
advdif_loss_correction = 1.0

In [ ]:
burgers_robustness = pd.read_csv('burgers_robustness.csv')
advdif_robustness = pd.read_csv('advdif_robustness.csv')

burgers_robustness['Data loss lowest (validation)'] = np.sqrt(burgers_robustness['Data loss lowest (validation)']/burgers_loss_correction)
advdif_robustness['Data loss lowest (validation)'] = np.sqrt(advdif_robustness['Data loss lowest (validation)']/advdif_loss_correction)


fig, ax = plt.subplots(1,1, figsize=(12,4), constrained_layout=True, facecolor='white')
fig.suptitle('Robustness analysis', fontsize=16)

ax.set_xlabel('Model', fontsize=12)
ax.set_ylabel('Relative error', fontsize=12)

def plot(df, ax, model):    
    x = df['alpha'].to_numpy()
    y = df['Data loss lowest (validation)'].to_numpy()
    ax.errorbar(model, np.mean(y[x == 0.0]), np.std(y[x == 0.0]), fmt='_--', elinewidth=2, capsize=15, ecolor='lightgrey', c='lightgrey', markeredgewidth=2, ms=20)
    ax.errorbar(model, np.mean(y[x != 0.0]), np.std(y[x != 0.0]), fmt='_--', elinewidth=2, capsize=15, ecolor='black', c='black', markeredgewidth=2, ms=20)
    #ax.scatter([model] * len(y[x == 0.0]), y[x == 0.0], c='lightgrey', s=5)
    #ax.scatter([model] * len(y[x != 0.0]), y[x != 0.0], c='black', s=5)

ax.set_xticks([0, 1, 2, 3])
ax.set_xticklabels(['Burgers', 'AdvDif', 'Waves', 'ShallowWater'])
plot(burgers_robustness, ax, 0)
plot(advdif_robustness, ax, 1)

legends = []
legends.append(plt.Line2D([0], [0], ls='', color='lightgrey', marker='.', ms=15, label='Without physics'))
legends.append(plt.Line2D([0], [0], ls='', color='black', marker='.', ms=15, label='With physics'))
ax.legend(handles=legends)